# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

Also import a plugin class
```csharp
public class StaticTextPlugin
{
    [KernelFunction, Description("Change all string chars to uppercase")]
    public static string Uppercase([Description("Text to uppercase")] string input) =>
        input.ToUpperInvariant();

    [KernelFunction, Description("Append the day variable")]
    public static string AppendDay(
        [Description("Text to append to")] string input,
        [Description("Value of the day to append")] string day) =>
        input + day;
}
```


In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

#!import plugins/StaticTextPlugin.cs
kernel.ImportPluginFromType<StaticTextPlugin>();

Define your ASK. What do you want the Kernel to do?

In [11]:
#pragma warning disable SKEXP0060

var ask = "Gumawa ka ng dalawang tula tungkol sa GinPom cocktail, tapos isalin mo sa English.";
var plan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine(plan);

{{!-- Step 1: Identify Key Values --}}
{{set "cocktailName" "GinPom"}}

{{!-- Step 2: Create Filipino Poems --}}
{{set "filipinoPoem1" (WriterPlugin-ShortPoem input="Pagmamahal sa " cocktailName)}}
{{set "filipinoPoem2" (WriterPlugin-ShortPoem input="Larawan ng " cocktailName)}}

{{!-- Step 3: Translate Poems to English --}}
{{set "englishPoem1" (WriterPlugin-Translate input=filipinoPoem1 language="English")}}
{{set "englishPoem2" (WriterPlugin-Translate input=filipinoPoem2 language="English")}}

{{!-- Step 4: Output the Result --}}
{{json (concat "English Translations of Filipino Poems about the " cocktailName " Cocktail\n\n" "Poem 1:\n" englishPoem1 "\n\n" "Poem 2:\n" englishPoem2)}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [12]:
#pragma warning disable SKEXP0060

var result = await plan.InvokeAsync(kernel);
result


English Translations of Filipino Poems about the GinPom Cocktail\n\nPoem 1:\nLove for food
There once was a girl named Jane,
Who had an insane love for food.
She would eat all day and night,
With all her strength,
And her stomach would never complain.\n\nPoem 2:\nThere was once a photo that was very impressive,
It showed a cat holding a brush in its paw.
The cat painted a work of art,
Using colors that never stopped,
And it became the most well-known in the country.